In [2]:
import pandas as pd

In [3]:
#read the date and show the table
df = pd.read_csv('../mvr-data.csv')
df

,DateOfRecord,DateOfEvent,Place,TypeOfRoad,Event,Vehicle1,Vehicle2,Outcome,AgeV1,AgeV2,TypeV2,Crossroad,Hour,GenderV1,GenderV2,AlchoholV2,Region,Mun
0,28.05.2015,28.05.2015,Тетово,Urban,кражба на велосипед,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Полошки,NaN
1,02.06.2015,02.06.2015,с.Враништа,Open,сообраќајна незгода,велосипед,автомобил,смрт,79.0,NaN,Фолксваген Пасат,не,11:00,NaN,NaN,NaN,Југозападен,Струга
2,30.08.2015,29.08.2015,Струга,Open,сообраќајна незгода,велосипед,автомобил,смрт,66.0,27.0,Мерцедес,не,16:00,NaN,NaN,NaN,Југозападен,NaN
3,16.09.2015,15.09.2015,Кичево,Open,сообраќајна незгода,велосипед,автомобил,смрт,74.0,44.0,Цитроен Берлинго,да,11:20,NaN,NaN,NaN,Југозападен,NaN
4,19.09.2015,18.09.2015,Скопје,Urban,сообраќајна незгода,велосипед,автомобил,смрт,13.0,47.0,Фиат Пунто,да,18:30,NaN,ж,NaN,Скопски,Центар
5,22.09.2015,21.09.2015,Свети Николе,Urban,напад на велосипедист,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17:45,NaN,NaN,NaN,Вардарски,NaN
6,26.09.2015,25.09.2015,Скопје,Urban,сообраќајна незгода,велосипед,автомобил,смрт,64.0,20.0,Ауди,да,14:00,NaN,NaN,NaN,Скопски,Центар
7,14.12.2015,13.12.2015,с.Долно Коњари,Rural,сообраќајна незгода,велосипед,автомобил,животна опасност,74.0,70.0,Опел Олимпија,не,15:30,NaN,NaN,NaN,Скопски,Петровец
8,01.01.2016,31.12.2015,Охрид,Urban,сообраќајна незгода,велосипед,автомобил,смрт,62.0,53.0,Фолксваген Пасат,не,17:00,NaN,NaN,да,Југозападен,NaN
9,21.02.2016,20.02.2016,с.Велешта,Open,сообраќајна незгода,велосипед со мотор,автобус,тешка повреда,52.0,45.0,Автобус,не,17:10,NaN,NaN,NaN,Југозападен,Струга


### R code for transformation

This should be rewritten in python:
this chunk makes a new df keeping only the traffic accinents (excluding other things like dog attacks), and adds new variables (columns) in order to have more fine-grained data points.


```
#keep the traffic accidents
traffacc <- accidents %>%
  #separate Date of event; having months as observations might be useful.
  separate(DateOfEvent, c("Day", "Month", "Year"), sep = "\\.", remove = FALSE) %>%
  #separate hours:mins, make numeric
  separate(Hour, c("H", "M"), sep = "\\:", remove = FALSE) %>%
  mutate(Hours = as.numeric(H)) %>%
  #filter so we keep just the traffic accidents
  filter(Event == "сообраќајна незгода") %>% 
  mutate(Outcome = ordered(Outcome, levels = c("непознато", "повреда", "тешка повреда", "животна опасност", "смрт")))
  
```